In [30]:
import joblib
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
from cleaning_data import clean_text_vietnamese
from __future__ import print_function
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from matplotlib.animation import FuncAnimation, PillowWriter
np.random.seed(17)

# Classify customer comments regrading sentiment
Problem: Classify Vietnamese customer comments by predicting whether the comments are positive or negative.

# Loading Phobert model

In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
bert_model = AutoModel.from_pretrained("vinai/phobert-base").to(device)

# Get bert embedding from sentence

In [70]:
def get_bert_embedding(sentence):
    sentence = clean_text_vietnamese(sentence, keep_punct=True)
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state
    sentence_embedding = embeddings.mean(dim=1).squeeze().cpu().numpy()
    return sentence_embedding

# Evaluate the accurate of model

In [51]:
def evaluate_model(test_file,svm_model):
    # Load test data
    test_data = pd.read_excel(test_file)
    
    # Ensure column names are lowercase
    test_data.columns = test_data.columns.str.lower()
    # Get predictions using lowercase column names
    X_test = [get_bert_embedding(text) for text in test_data['text']]
    y_true = test_data['label']
    y_pred = svm_model.predict(X_test)
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred)
    
    print(f"\nAccuracy: {accuracy:.6f}")
    print("\nDetailed Classification Report:")
    print(report)
    
    return accuracy, report

# Loading the pre-trained model

In [52]:
model_path_poly = 'svm_model_poly.pkl'
model_path_linear = 'svm_model_linear.pkl'
model_path_rbf = 'svm_model_rbf.pkl'
model_path_logistic_regression = 'model_logistic.pkl'
model_path = {
    "Linear SVM": model_path_linear,
    "Polynomial SVM": model_path_poly,
    "RBF SVM": model_path_rbf,
    "Logistic Regression": model_path_logistic_regression
}

In [53]:
results = []
for model_name, model_file in model_path.items():
    model = joblib.load(model_file)
    test_file = "Testing_dataset.xlsx"
    print(f"Evaluating {model_name}:")
    accuracy, report = evaluate_model(test_file, model)
    results.append({
        'Model Name': model_name,
        'Model Type': "SVM" if "SVM" in model_name else "Logistic Regression",
        'Accuracy (%)': f"{accuracy * 100:.4f}"
    })
    print("\n")

Evaluating Linear SVM:

Accuracy: 0.980490

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.98      0.97       585
           1       0.99      0.98      0.99      1414

    accuracy                           0.98      1999
   macro avg       0.97      0.98      0.98      1999
weighted avg       0.98      0.98      0.98      1999



Evaluating Polynomial SVM:

Accuracy: 0.982991

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       585
           1       1.00      0.98      0.99      1414

    accuracy                           0.98      1999
   macro avg       0.97      0.98      0.98      1999
weighted avg       0.98      0.98      0.98      1999



Evaluating RBF SVM:

Accuracy: 0.983492

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       585
    

In [50]:
comparison_df = pd.DataFrame(results)

# Display a pretty table using tabulate
print("\nModel Comparison Table:")
print(tabulate(comparison_df, headers='keys', tablefmt='fancy_grid'))


Model Comparison Table:
╒════╤═════════════════════╤═════════════════════╤════════════════╕
│    │ Model Name          │ Model Type          │   Accuracy (%) │
╞════╪═════════════════════╪═════════════════════╪════════════════╡
│  0 │ Linear SVM          │ SVM                 │        98.049  │
├────┼─────────────────────┼─────────────────────┼────────────────┤
│  1 │ Polynomial SVM      │ SVM                 │        98.2991 │
├────┼─────────────────────┼─────────────────────┼────────────────┤
│  2 │ RBF SVM             │ SVM                 │        98.3492 │
├────┼─────────────────────┼─────────────────────┼────────────────┤
│  3 │ Logistic Regression │ Logistic Regression │        98.1491 │
╘════╧═════════════════════╧═════════════════════╧════════════════╛


# Using RBF kernel for demo

In [38]:
svm_model = joblib.load('svm_model_rbf.pkl')

In [40]:
while True:
    input_text = input("Nhập câu để kiểm tra cảm xúc (hoặc nhập 'q' để thoát): ")
    if input_text == 'q':
            break
    sentence_embedding = get_bert_embedding(input_text)
    predicted_sentiment = svm_model.predict([sentence_embedding])[0]  # Dự đoán 1 câu
    print(input_text)
    if predicted_sentiment == 0:
            print("Cảm xúc: Tiêu cực")
    else:    
            print("Cảm xúc: Tích cực")

sản phẩm này tệ thế
Cảm xúc: Tiêu cực
sản phẩm này không tệ
Cảm xúc: Tích cực
sản phẩm này dùng đã thật, tôi rất thích nó
Cảm xúc: Tích cực
nếu biết trước công dụng của nó tuyệt vời đến vậy tôi đã mua từ lâu
Cảm xúc: Tích cực
sản phẩm này dùng rất tệ, dùng được có vài ngày đã hư lên hư xuống, thật đáng thất vọng
Cảm xúc: Tiêu cực
sản phẩm này không tốt
Cảm xúc: Tiêu cực
sản phẩm này tốt
Cảm xúc: Tích cực
